In [9]:
from importlib import reload 
import sys
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial.distance import braycurtis
from scipy.special import softmax
import seaborn as sns
# # adding Folder_2 to the system path
# sys.path.insert(0, r'C:\Users\tomer\Desktop\BSc\year3\sem B\workshop_microbiome\code')
# # sys.path.insert(0, r'C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\code')

from imports import  *

import model_class
reload(model_class)


<module 'model_class' from 'c:\\Users\\yuvald\\Documents\\Uni\\סמסטר ב\\workshop_microbiome\\Milestone2\\model_class.py'>

$$f(D,\Delta_t) = \alpha e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right) D_{t-1} + \beta\left(1- e^{-\lambda \Delta_t} \cdot \cos\left(\frac{2 \pi \Delta_t}{365}\right)\right)\cdot \overline{\{D_1,...,D_{t-2}\}}$$

In [10]:
import model_class
reload(model_class)


<module 'model_class' from 'c:\\Users\\yuvald\\Documents\\Uni\\סמסטר ב\\workshop_microbiome\\Milestone2\\model_class.py'>

In [11]:
data_path = r"train_data.csv"
metadata_path = r"train_metadata.csv"

In [3]:
metadata_df = pd.read_csv(metadata_path)

In [5]:
metadata_df.columns

Index(['sample', 'baboon_id', 'collection_date', 'sex', 'age', 'social_group',
       'group_size', 'rain_month_mm', 'season', 'hydro_year', 'month',
       'diet_PC1', 'diet_PC2', 'diet_PC3', 'diet_PC4', 'diet_PC5', 'diet_PC6',
       'diet_PC7', 'diet_PC8', 'diet_PC9', 'diet_PC10', 'diet_PC11',
       'diet_PC12', 'diet_PC13'],
      dtype='object')

In [6]:
metadata_df.social_group.unique()

array(['g_1.22', 'g_2.2', 'g_1.1', 'g_1.12', 'g_1.21', 'g_1.211',
       'g_1.11', 'g_2.1', 'g_2.12', 'g_2.11'], dtype=object)

In [7]:
# data_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_data.csv", index_col="sample")
# metadata_df = pd.read_csv(r"C:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\train_metadata.csv",  index_col="sample")

In [21]:
super_model = model_class.superModel(data_path, metadata_path)

c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:227: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_clean["collection_date"] = (pd.to_datetime(metadata_clean['collection_date']) - pd.Timestamp('1970-01-01')).dt.days


In [ ]:
import time
start = time.time()
super_model.fit()
time.time() - start

In [22]:
for baboon in super_model.baboons:
    baboon.alpha_ = np.zeros([61,61])
    baboon.beta_ = np.eye(61,61)

In [23]:
baboons = super_model.baboons
super_model.baboons = super_model.baboons[:60]

In [6]:
Baboon_96 = baboons[-1]

In [7]:
last_data = Baboon_96.data[57:]
Baboon_96.data = Baboon_96.data[:57]

In [8]:
res = super_model.predict(Baboon_96.data, Baboon_96.metadata, iterative=True)

c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:149: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:149: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:149: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c

In [14]:
last_data.iloc[1].values

array([5.44307300e-03, 5.06171100e-03, 1.80280100e-03, 3.53626400e-03,
       2.62099570e-02, 4.81209264e-01, 7.63000000e-04, 1.62945500e-03,
       9.36000000e-04, 3.48772710e-02, 6.93000000e-04, 1.39370410e-02,
       1.59478600e-03, 8.07793600e-03, 5.27666070e-02, 8.11260600e-03,
       1.35210100e-03, 1.04000000e-04, 1.51504650e-02, 2.04548600e-03,
       1.73346300e-03, 0.00000000e+00, 6.93000000e-04, 1.56011600e-03,
       1.04007800e-03, 0.00000000e+00, 1.14408500e-03, 1.12328390e-02,
       1.52544700e-03, 2.80821000e-03, 5.89000000e-04, 5.13104980e-02,
       3.81000000e-04, 1.22729160e-02, 0.00000000e+00, 1.56358340e-02,
       2.94688700e-03, 1.78130634e-01, 1.20995700e-02, 2.66953300e-03,
       2.73887100e-03, 7.10719700e-03, 0.00000000e+00, 1.38677000e-03,
       8.32000000e-04, 0.00000000e+00, 0.00000000e+00, 2.11482500e-03,
       4.54167200e-03, 0.00000000e+00, 5.55000000e-04, 0.00000000e+00,
       4.16000000e-04, 1.73000000e-04, 3.81361800e-03, 5.55000000e-04,
      

In [26]:
Baboon_78 = baboons[-2]

In [27]:
baboon_78_data = Baboon_78.data[2:]
Baboon_78.data = Baboon_78.data[:2]

In [38]:
baboon_101 = baboons[0]

In [28]:
len(baboon_78_data)

111

In [40]:
baboon_101.predict(baboon_78_data, Baboon_78.metadata, weights=1/6,lambda_=super_model.lambda_, iterative=True)

array([1.24057491e-03, 3.15650183e-04, 3.69157831e-04, 1.14752658e-03,
       3.03616709e-03, 5.73771146e-02, 3.85644062e-04, 4.78053300e-03,
       1.43938614e-04, 1.19478511e-02, 1.39236755e-03, 1.97681965e-03,
       2.62940892e-04, 5.37486465e-04, 5.89005831e-03, 1.81525931e-03,
       3.81987975e-04, 6.79115403e-04, 1.07761684e-03, 1.63301815e-03,
       9.19532382e-04, 2.25564882e-04, 3.44338491e-04, 1.90573941e-04,
       9.44908897e-05, 1.76799823e-04, 1.28423014e-04, 3.36367188e-03,
       4.01221637e-04, 2.91756873e-04, 2.97343118e-04, 1.02942657e-02,
       3.24110522e-04, 6.24239161e-04, 8.29578039e-05, 4.94252632e-04,
       8.99166781e-04, 1.86300538e-02, 1.33671929e-03, 3.96180162e-03,
       8.39612948e-04, 1.27707178e-02, 3.82091164e-04, 7.70551110e-04,
       4.39546536e-04, 3.38491798e-04, 9.70834818e-05, 9.90834607e-04,
       6.07926897e-04, 2.15593091e-04, 1.31381241e-04, 9.72255037e-05,
       2.97692443e-04, 3.18015639e-03, 1.19186211e-03, 6.01236008e-04,
      

In [16]:
iterative_res = super_model.predict(Baboon_78.data, Baboon_78.metadata, iterative=True)
noninterative_res = super_model.predict(Baboon_78.data, Baboon_78.metadata, iterative=False)

c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c:\Users\yuvald\Documents\Uni\סמסטר ב\workshop_microbiome\Milestone2\model_class.py:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  known_data = known_data.append(pd.Series(predictions[j-n], index = known_data.columns), ignore_index = True)
c

In [37]:
Baboon_78.metadata

,baboon_id,collection_date,sex,age,social_group,group_size,rain_month_mm,season,hydro_year,month,...,diet_PC4,diet_PC5,diet_PC6,diet_PC7,diet_PC8,diet_PC9,diet_PC10,diet_PC11,diet_PC12,diet_PC13
sample,,,,,,,,,,,,,,,,,,,,,
sample_11407-TTACCACATCTA-394,Baboon_78,11659,M,1.117043,g_1.21,22,13.2,wet,2002,12,...,-7.299950,11.585974,-8.259149,-6.951492,13.476441,8.591452,-2.715730,-0.670181,-0.363471,-0.270125
sample_12051-GTGCGCCTTCTG-408,Baboon_78,11725,M,1.297741,g_1.21,21,61.0,wet,2002,2,...,-16.448277,-14.826011,-2.471410,4.679189,0.180357,7.190213,5.145853,-8.074240,0.534689,0.466634
sample_11408-ATTAATGAAGCG-395,Baboon_78,11729,M,1.308693,g_1.21,21,61.0,wet,2002,2,...,-13.131171,-15.033754,-1.162481,3.358549,0.529433,5.814366,3.295151,-11.286747,0.469203,0.410511
sample_12050-CAGTAGCGATAT-407,Baboon_78,11740,M,1.338809,g_1.21,21,2.0,wet,2002,2,...,-10.538072,-18.027935,-4.024002,-3.760225,1.617033,2.790060,-0.988068,-8.916010,0.414878,0.198270
sample_11407-TCAATACAATAT-394,Baboon_78,11834,M,1.596167,g_1.21,24,51.0,wet,2002,5,...,7.244653,-5.417804,-2.619896,-3.011515,-2.509070,4.895551,-2.253855,-2.144438,0.211195,0.092890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_11407-AGGGACTTCAAT-394,Baboon_78,15045,M,10.387410,g_2.2,113,31.0,wet,2011,3,...,-0.887084,-0.887129,1.793682,2.890806,0.489460,3.894504,1.639758,-0.830671,0.445997,-0.001310
sample_11410-GTAGCACTCATG-396,Baboon_78,15118,M,10.587270,g_2.2,114,3.2,wet,2011,5,...,-1.796998,1.102359,-0.692042,0.505585,1.852034,0.843750,-0.483853,-0.338856,0.562867,0.113109
sample_12049-GTTAACTTACTA-407,Baboon_78,15188,M,10.778920,g_2.2,116,0.0,dry,2011,8,...,-3.741875,3.188046,-3.503696,2.028410,-1.355503,-1.415436,-0.842528,-0.896124,0.455143,0.085074


In [29]:
len(iterative_res)

111

In [30]:
braycurtis_78_iterative = 0
for i in range(111):
    braycurtis_78_iterative += braycurtis(iterative_res[i], baboon_78_data.iloc[i].values)

braycurtis_78_non_iterative = 0
for i in range(111):
    braycurtis_78_non_iterative += braycurtis(noninterative_res[i], baboon_78_data.iloc[i].values)

In [44]:
for i in noninterative_res.flatten():
    print(i)

0.01197941189274708
0.0
0.006832008079457276
0.020683204267946273
0.0691623879276573
0.22592422869602816
0.0
0.09312121271315163
0.0012166591922321522
0.07927001752466276
0.032569027145906294
0.0013102482070192166
0.003649977576696457
0.0028076744436125616
0.06448293018830298
0.03313056223462883
0.0
0.013289660099766298
0.006644830049883149
0.002152550340102954
0.0
0.0017781942809546962
0.0028076744436125616
0.0017781942809546962
0.0
0.00159101525138041
0.0
0.009358915478708648
0.0007490001183420186
0.0006550001034900162
0.0006550001034900162
0.013944784203275905
0.0008420001330360212
0.00159101525138041
0.0009360001478880234
0.002714085428825498
0.007955078256902363
0.05849322424192943
0.03153954598324827
0.0
0.0184370639130561
0.08713150676677808
0.005802527916799412
0.004117922650631781
0.005334581842863932
0.003649977576696457
0.0019653723105288253
0.030229297776229045
0.0
0.010949931730089213
0.0013102482070192166
0.0
0.0031820315027609775
0.010014039582218253
0.011137109759663344

In [15]:
x = 0
for i in range(10):
    x += braycurtis(res[i], last_data.iloc[i].values)

In [16]:
x

3.056247151132685

In [17]:
non_iter_res = super_model.predict(Baboon_96.data, Baboon_96.metadata, iterative=False)

In [18]:
y = 0
for i in range(10):
    y += braycurtis(non_iter_res[i], last_data.iloc[i].values)

In [19]:
y

3.000219516873837

In [11]:
baboon_101 = super_model.baboons[0]

In [12]:
baboon_101.fit(0)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[ 9.99560751e-01, -7.52734711e-05, -6.95880161e-05, ...,
         -2.14871154e-05, -1.26788573e-04, -5.54778395e-05],
        [-2.42678641e-04,  9.99945119e-01, -1.70861341e-05, ...,
         -1.49847119e-05, -6.01530891e-05, -4.34695731e-05],
        [-7.14703165e-05, -7.50187236e-05,  9.99941819e-01, ...,
         -1.52833813e-05, -1.21462790e-04, -7.36542145e-05],
        ...,
        [-2.84972081e-04, -5.47160373e-05, -1.75448747e-05, ...,
          9.99983832e-01, -6.24160466e-05, -4.34661569e-05],
        [-2.67673170e-04, -5.78305930e-05, -4.19201036e-05, ...,
         -1.54009945e-05,  9.99908403e-01, -2.34738525e-05],
        [-1.73669419e-04, -4.10358079e-05, -2.36890702e-05, ...,
         -1.13089311e-05, -6.84743502e-05, 

In [13]:
# calculate the objective function

'''for the ith row in self.data (from row 3)
1. calculate mean of previous i-2 rows = d_mean
2. calculate time difference between ith row and i-1th row = d_time
calculate the prediction for the ith row using the formula'''


'''
calculate difference between prediction and actual value using bray-curtis dissimilarity and return the sum/mean -TBD'''
self = baboon_101
lambda_ = 0
method="counts"
alpha = baboon_101.alpha_
beta = baboon_101.beta_


D_t1 = self.transformed_data[1:-1].values
D_mean = self.df_cumulative_mean[:-2].values
cos = 0# np.cos((2*np.pi*self.delta_t[2:].values)/365)
exp = np.exp(-lambda_*self.delta_t[2:].values)

f = alpha@(exp*cos*D_t1.T) + beta@((1-exp*cos)*D_mean.T)

f = to_composition(f.T, type = method) # transpose f to match the shape of D - each row is a sample

# calculate bray-curtis dissimilarity
bc =  np.array([braycurtis(self.data.values[i+2], f[i]) for i in range(len(f))])

f.shape

(51, 61)